In [2]:
import numpy as np
import pandas as pd 
from tqdm import tqdm
import random

import torch 
from torch.utils.data import DataLoader, Dataset, random_split
import torch.nn as nn 
from torch import optim
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
import torch.nn.functional as F

from joblib import Parallel, delayed
from Evolution.EvolutionaryLib import BaseChromosome, ChromosomeClassFactory


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
class MyDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, item):
        return self.data[item][0], self.data[item][1]

classes = {0, 1, 2, 3, 4, 5}
train_size = 3000
test_size = 100 
train_batch_size = 16
test_batch_size = 32

In [4]:
transformer = transforms.Compose([
    transforms.ToTensor(),
    transforms.CenterCrop(24)
])

train_data = datasets.MNIST(
    root = 'data',
    train = True,
    transform = transformer,
    download = True,
)
test_data = datasets.MNIST(
    root = 'data',
    train = False,
    transform = transformer
)

train_indices = torch.cat([torch.where(train_data.targets == c)[0][:train_size] for c in classes])
test_indices = torch.cat([torch.where(test_data.targets == c)[0][:test_size] for c in classes])

train_data = torch.utils.data.Subset(train_data, train_indices)
test_data = torch.utils.data.Subset(test_data, test_indices)

train_dataset = MyDataset(train_data)
test_dataset = MyDataset(test_data)

train_loader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False)

In [5]:
class CNN(nn.Module): 
    def __init__(self, kernel_structure: tuple[int], classifier_structure: tuple[int], out_classes: int): 
        super(CNN, self).__init__()
        self.image_size = (24, 24)
        self.kernel_structure = kernel_structure
        self.classifier_structure = classifier_structure
        cropped_img = (self.image_size[0] - sum(kernel_structure) + len(kernel_structure), 
                       self.image_size[1]  - sum(kernel_structure) + len(kernel_structure))
        self.cropped_img = cropped_img
        self.channels = 3

        # Convolutional part initialisation 
        architecture = [nn.Conv2d(in_channels=1, out_channels=self.channels, kernel_size=kernel_structure[0]), nn.ReLU()]
        for kernel_size in kernel_structure[1:]: 
            architecture.append(nn.Conv2d(in_channels=self.channels, out_channels=self.channels, kernel_size=kernel_size))
            architecture.append(nn.ReLU())

        self.convolve = nn.Sequential(
            *architecture
        )

        # CLassifier part initialisation 
        architecture = [nn.Linear(self.channels * cropped_img[0] * cropped_img[1], classifier_structure[0]), nn.ReLU()]
        for i in range(len(classifier_structure) - 1): 
            architecture.append(nn.Linear(classifier_structure[i], classifier_structure[i+1]))    
            architecture.append(nn.ReLU())

        architecture.append(nn.Linear(classifier_structure[-1], out_classes))

        self.classifier = nn.Sequential(
            *architecture
        )

    def forward(self, x):
        x = self.convolve(x)
        x = self.classifier(x.view(-1, self.channels * self.cropped_img[0] * self.cropped_img[1]))
        return F.log_softmax(x, dim=1)
criterion = nn.CrossEntropyLoss()

In [6]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()

    for _ in range(epoch): 
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
    
            output = model(data).softmax(dim=1)
    
            loss = criterion(output, target)
            
            loss.backward()
            optimizer.step()

def test(model, device, test_loader):
    model.eval()
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    return 100. * correct / len(test_loader.dataset) 

In [7]:
class MyChromosome(BaseChromosome):
    def __init__(self):
        super().__init__()

    def crossover(self, other):
        new = MyChromosome()
        self.copy(new)

        new['kernel_structure'] = self._kernel_structure.crossover(other._kernel_structure)
        new['classifier_structure'] = self._classifier_structure.crossover(other._classifier_structure)

        new['lr'] = (self['lr'] + other['lr']) / 2
        new['epochs'] = random.choice([self['epochs'], other['epochs']])

        return new

    def mutate(self, rate=0.3):
        new = MyChromosome()
        self.copy(new)

        if random.random() < rate:
            new.kernel_structure = new._kernel_structure.mutate()
        else:
            new.kernel_structure = self.kernel_structure

        if random.random() < rate:
            new.classifier_structure = new._classifier_structure.mutate()
        else:
            new.classifier_structure = self.classifier_structure
        

        new.epochs = new._epochs.get() if random.random() < rate else self.epochs
        new.lr = new._lr.get() if random.random() < rate else self.lr
        
        return new

chromosome_factory = ChromosomeClassFactory(
        kernel_structure={'n':[2, 3, 4], 'range': [3, 5]}, 
        classifier_structure={'n': [1, 2, 3], 'range': list(range(20, 80))}, 
        lr=(1e-7, 1), 
        epochs=list(range(2, 10))
)

In [8]:
import torch.optim as optim
from math import log

def evaluate(chromosome, show_metric=False): 
    n_iterations = 2
    metric = 0

    for _ in range(n_iterations): 
        model = CNN(chromosome.kernel_structure, chromosome.classifier_structure, len(classes)).to(device)
        optimizer = optim.Adam(model.parameters(), lr=chromosome.lr)
        
        train(model, device, train_loader, optimizer, chromosome.epochs)
        metric += test(model, device, test_loader)
        
    metric /= n_iterations
    
    if show_metric: 
        print(f'{metric=}')
        
    return metric

In [9]:
# evolution hyperparameters initialisation 
POPULATION_SIZE = 30
ITERATIONS = 40
best = None

In [12]:
# model = CNN((5, 5, 3, 3), (30, ), len(classes)).to(device)
# optimizer = optim.Adam(model.parameters(), lr=0.01)
# # print(model)

# train(model, device, train_loader, optimizer, 7)
# test(model, device, test_loader)

In [10]:
population = []
for _ in range(POPULATION_SIZE):
    c = chromosome_factory.generate(MyChromosome)
    population.append(c)

with Parallel(n_jobs=2) as parallel: 
    for iteration in tqdm(range(ITERATIONS)):
        # scores = [evaluate(chromosome) for chromosome in population]
        scores = parallel(delayed(evaluate)(chromosome) for chromosome in population)
        for i, chromosome in enumerate(population):
            chromosome.set_score(scores[i])
    
    
        # selection of best instances 
        population = sorted(population, key=lambda x: x.get_score(), reverse=True)[:POPULATION_SIZE // 3]
        # checking for the best solution 
        if best is None or population[0].get_score() > best.get_score():
            best = population[0]
            print(iteration)
            print(best)
    
        # filling the population with new genes 
        while len(population) < POPULATION_SIZE:
            r = random.random()
            if r < 0.33:
                population.append(random.choice(population).crossover(random.choice(population)))
            elif 0.33 <= r < 0.66:
                population.append(random.choice(population).mutate())
            else:
                population.append(chromosome_factory.generate(MyChromosome))
    else: 
        # scores = [evaluate(chromosome) for chromosome in population]
        scores = parallel(delayed(evaluate)(chromosome) for chromosome in population)
        for i, chromosome in enumerate(population):
            chromosome.set_score(scores[i])

  0%|                                                                                           | 0/40 [00:00<?, ?it/s]C:\Users\pc\AppData\Local\Programs\Python\Python311\Lib\site-packages\joblib\externals\loky\process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
  2%|██                                                                              | 1/40 [15:14<9:54:44, 914.99s/it]

0
--Chromosome--
score: 17.416666666666668
kernel_structure: (5, 5)
classifier_structure: (46, 25)
lr: 0.7648446769334614
epochs: 5



  5%|████                                                                            | 2/40 [29:39<9:20:42, 885.33s/it]

1
--Chromosome--
score: 16.666666666666668
kernel_structure: (5, 3, 3, 5)
classifier_structure: (64, 65, 79)
lr: 0.694383841328918
epochs: 4



  8%|██████                                                                          | 3/40 [42:47<8:38:37, 841.02s/it]

...

37
--Chromosome--
score: 97.16666666666666
kernel_structure: (3, 3)
classifier_structure: (40,)
lr: 0.0060438620544241055
epochs: 2



 98%|█████████████████████████████████████████████████████████████████████████████  | 39/40 [9:23:59<12:16, 736.48s/it]

38
--Chromosome--
score: 97.25
kernel_structure: (3, 3)
classifier_structure: (43,)
lr: 0.0060103161755861115
epochs: 2



100%|███████████████████████████████████████████████████████████████████████████████| 40/40 [9:37:24<00:00, 866.10s/it]

39
--Chromosome--
score: 98.58333333333334
kernel_structure: (5, 5)
classifier_structure: (55,)
lr: 0.0017810669092231872
epochs: 9



In [12]:
for ch in population: 
    if ch.get_score() is None: 
        ch.set_score(0)

for ch in sorted(population, key=MyChromosome.get_score, reverse=True): 
    print(ch)

--Chromosome--
score: 98.75
kernel_structure: (5, 5)
classifier_structure: (55,)
lr: 0.0017810669092231872
epochs: 9

--Chromosome--
score: 98.75
kernel_structure: (5, 5)
classifier_structure: (55,)
lr: 0.0017810669092231872
epochs: 9

--Chromosome--
score: 97.75
kernel_structure: (3, 5)
classifier_structure: (40,)
lr: 0.006035475584714607
epochs: 3

--Chromosome--
score: 96.66666666666667
kernel_structure: (5, 3)
classifier_structure: (40,)
lr: 0.0060103161755861115
epochs: 3

--Chromosome--
score: 96.25
kernel_structure: (5, 3)
classifier_structure: (40,)
lr: 0.0060103161755861115
epochs: 3

--Chromosome--
score: 96.0
kernel_structure: (3, 3)
classifier_structure: (40,)
lr: 0.0060103161755861115
epochs: 2

--Chromosome--
score: 95.58333333333333
kernel_structure: (3, 3)
classifier_structure: (43,)
lr: 0.006022895880150359
epochs: 2

--Chromosome--
score: 95.0
kernel_structure: (3, 5)
classifier_structure: (43,)
lr: 0.005980963531602867
epochs: 2

--Chromosome--
score: 94.083333333333

In [43]:
import pickle 
# TODO: make parameter distribution 



ps = None 
for i, ch in enumerate(population): 
    ps = pickle.dumps(ch)

    with open(f'chromosomes/{i}.txt', mode='wb') as file: 
        ps = pickle.dumps(ch)
        file.write(ps)

In [44]:
for i in range(30): 
    ch_loaded = pickle.loads(open(f'chromosomes/{i}.txt', mode='rb').read())
    print(ch_loaded)

--Chromosome--
score: 98.75
kernel_structure: (5, 5)
classifier_structure: (55,)
lr: 0.0017810669092231872
epochs: 9

--Chromosome--
score: 91.33333333333333
kernel_structure: (3, 5)
classifier_structure: (40,)
lr: 0.006024992497577734
epochs: 3

--Chromosome--
score: 54.41666666666667
kernel_structure: (3, 3)
classifier_structure: (40,)
lr: 0.006035475584714607
epochs: 2

--Chromosome--
score: 55.75
kernel_structure: (3, 3)
classifier_structure: (43,)
lr: 0.006027089115005109
epochs: 2

--Chromosome--
score: 96.0
kernel_structure: (3, 3)
classifier_structure: (40,)
lr: 0.0060103161755861115
epochs: 2

--Chromosome--
score: 95.58333333333333
kernel_structure: (3, 3)
classifier_structure: (43,)
lr: 0.006022895880150359
epochs: 2

--Chromosome--
score: 88.66666666666666
kernel_structure: (3, 5)
classifier_structure: (40,)
lr: 0.006022895880150359
epochs: 3

--Chromosome--
score: 95.0
kernel_structure: (3, 5)
classifier_structure: (43,)
lr: 0.005980963531602867
epochs: 2

--Chromosome--
s